```{code-block}
:class: thebe, thebe-init
# Automatic import for live code
import numpy as np
from shrbk.plot import *
```

 (sec:measuring-bias)=
# Measuring Bias

In this section we will discuss several methods to measure shear bias.


## Shear bias from individual galaxies

{cite}`PKSB17` present a shape-noise-free shear bias estimator from individual simulated galaxies. The principle is similar to the metacalibration technique: A small shear is applied to a galaxy image, and the response matrix of the measured ellipticity to the shear is determined by finite differences.

In general, the shear estimate for a single galaxy is dominated by shape noise. To beat down this noise, traditional methods use a very large number of galaxies. Some additional improvement can be gained by using rotated versions of the same galaxy, such that the sum of the intrinsic ellipticities is zero. The resulting shear estimate has a reduced variance.

However, this does not account for the variance from the measured ellipticity, which does not only depend on the intrinsic one, but on the PSF, pixel noise, etc. In addition, all versions of the rotated galaxy need to be detected and have an measured shape, and therefore, selection biases are difficult to quantify with this method.

To reduce shape noise (to virtually zero), {cite}`PKSB17` uses the same noise realisation is used for each sheared, and the unsheared image. The resulting estimate of the response per galaxy is highly precise. It is still not very accurate, since the measured value depends on the noise realisation. To increase the accuracy, a large number of simulated galaxies with different noise realisations is required. These galaxies are however anyway simulated to cover the large parameter space of galaxy properties.

## Example
Here we compute the individual shear bias from image simulations. This method is implemented in the library `shear_bias`.

In [ ]:
# Load the libraries
import shear_bias as sb
import galsim
import matplotlib.pylab as plt
import numpy as np

%matplotlib inline

### Set up

#### Shear values

First, set the value of the small shear $\Delta g$ to add to the galaxy images.

In [ ]:
dg = 0.02

Define the five steps, one in each positive and negative direction along the two coordinate axis, plus the original unchanged image (0, 0).

In [ ]:
g_steps = [(-1, 0), (0, -1), (1, 0), (0, 1), (0, 0)]

Create dictionary of shear values with step tuples as keys.

In [ ]:
g_values = {}
for step in g_steps:
    g_values[step] = (step[0] * dg, step[1] * dg)

#### Number of galaxy images

In [ ]:
n_gal = 10

### Downloading HST COSMOS galaxy images

In [ ]:
data_dir = 'data'
cat_file_name = 'real_galaxy_catalog_23.5_example.fits'

sky_level = 1e5        # ADU / arcsec^2
gal_flux = 1e5   # arbitrary, choose large value for not too noisy images

pixel_scale = 0.16  # arcsec
random_seed = 5693562491

In [ ]:
sb.download_HST_images(dest_dir=data_dir)

In [ ]:
real_galaxy_catalog = galsim.RealGalaxyCatalog(cat_file_name, dir=data_dir)

### PSF

In [ ]:
# Create PSF
psf = galsim.Gaussian(fwhm=fwhm_psf[0], flux=flux_frac_psf[0])
for i in range(1, len(fwhm_psf)):
    psf = psf + galsim.Gaussian(fwhm=fwhm_psf[i], flux=flux_frac_psf[i])

# Draw PSF
psf_image = psf.drawImage(scale=pixel_scale)

### Galaxies

In [ ]:
# Lensing magnification. Set to 1 for no magnification.
magnification = 1

# Constant background
background = sky_level * pixel_scale**2

In [ ]:
all_images = {}
im_test = {}
k_test = 3

for step in g_steps:
    all_images[step] = []

for k in range(n_gal):

    print(k, end=' ')
    rng = galsim.UniformDeviate(random_seed+k+1)
    gal = galsim.RealGalaxy(real_galaxy_catalog, index=k, flux=gal_flux)
    
    for step in g_steps:
        gal_sh = gal.shear(g1=g_values[step][0], g2=g_values[step][1])

        if magnification != 1:
            gal_sh = gal_sh.magnify(magnification)
        
        final = galsim.Convolve([psf, gal_sh])
        
        dx = rng() - 0.5
        dy = rng() - 0.5

        if k == 0:
            im = final.drawImage(scale=pixel_scale, offset=(dx, dy))
            xsize, ysize = im.array.shape
        else:
            im = galsim.ImageF(xsize, ysize)
            final.drawImage(im, scale=pixel_scale, offset=(dx, dy))
 
        if k == k_test and step == (0, 0):
            im_test['no_noise'] = np.copy(im.array)

        im.addNoise(galsim.PoissonNoise(rng=rng, sky_level=sky_level))

        #ccd_noise = galsim.CCDNoise(rng, sky_level=0., gain=1., read_noise=0.)
        #im.addNoise(ccd_noise)
        if k == k_test:
            if step == (0, 0):
                im_test['no_shear'] = np.copy(im.array)
            if step == (1, 0):
                im_test['+g1'] = np.copy(im.array)
            if step == (0, 1):
                im_test['-g1'] = np.copy(im.array)

        all_images[step].append(im)
print()

im_test['+g1-g1'] = all_images[(1,0)][k_test].array - all_images[(0, 1)][k_test].array

#### Plot some galaxies

In [ ]:
nim = len(im_test)
fig, (axes) = plt.subplots(nrows=1, ncols=nim, figsize=(10, 5), tight_layout=True)

for ax, key in zip(axes, im_test):
    ax.imshow(im_test[key])
    ax.set_title(key)